In [1]:
import pandas as pd
import numpy as np
from sklearn.covariance import GraphicalLasso 

In [2]:
GTEx_dir = "/data/abattle4/prashanthi/recount3_paper/data/GTEx/corrected_expression/"
select_studies_GTEx = ["Adipose_Subcutaneous", "Cells_Cultured_fibroblasts", "Heart_Left_Ventricle", "Artery_Aorta", "Colon_Sigmoid"]

In [3]:
# Read network
resDir = "/data/abattle4/prashanthi/recount3_paper/results/"
aggType = "weighted_cov_networks"
dat_name = "sra_normal"
nStudies = "100"

In [4]:
lambda_vec = []
for lambda_p in np.arange(0.08, 1.02, 0.02):
    lambda_vec.append("{:.2f}".format(lambda_p))
res_ll = {'Lambda': lambda_vec}
res_ll = pd.DataFrame(res_ll, columns = ['Lambda'])

In [5]:
for i in range(len(select_studies_GTEx)):
    loglik = []
    for lambda_p in np.arange(0.08, 1.02, 0.02):
        net_fn = resDir + aggType + "/" + dat_name + "/net_" + nStudies + "/lambda_" + "{:.2f}".format(lambda_p) + ".txt"
        net = pd.read_csv(net_fn, delimiter = ",", index_col = 0)
        X_test = pd.read_csv(GTEx_dir + select_studies_GTEx[i] + ".txt", delimiter = ",", index_col = 0)
        common_genes = net.index.intersection(X_test.index)
        net = net.loc[common_genes]
        X_test = X_test.loc[common_genes]
        net = net.to_numpy()
        X_test = X_test.to_numpy()
        X_test = X_test.T
        estimator = GraphicalLasso()
        estimator.precision_ = net
        estimator.location_ = X_test.mean(0)
        loglik.append(estimator.score(X_test)*X_test.shape[0])
    res_ll[select_studies_GTEx[i]] = loglik

In [6]:
res_ll['mean'] = res_ll.iloc[:, 1:5].mean(axis=1)